In [1]:
# Imports
%load_ext autoreload
%autoreload 2

# ignore warnings for readability
import warnings
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import synthesizer
from synthesizer.parametric import Galaxy
from synthesizer.emission_models.attenuation import Inoue14
from synthesizer.emission_models import PacmanEmission, TotalEmission, EmissionModel, IntrinsicEmission, StellarEmissionModel, STELLAR_MODELS, IncidentEmission
from synthesizer.emission_models.attenuation import PowerLaw
from synthesizer.emissions import plot_spectra
from synthesizer.emission_models.dust.emission import Greybody
from synthesizer.grid import Grid
from synthesizer.parametric import SFH, Stars, ZDist
from synthesizer import check_openmp
from synthesizer.instruments import Instrument, FilterCollection

from typing import Dict, Any, List, Tuple, Union, Optional, Type
from abc import ABC, abstractmethod
import copy
from scipy.stats import uniform, loguniform
from astropy.io import ascii
from unyt import unyt_array, unyt_quantity, erg, cm, s, Angstrom, um, Hz, m, nJy, K, Msun, Myr, yr, Unit, kg
from unyt.equivalencies import SpectralEquivalence
from astropy.cosmology import Planck18, Cosmology, z_at_value
import astropy.units as u
from matplotlib.ticker import ScalarFormatter, FuncFormatter
from tqdm import tqdm
from ltu_ili_testing import (generate_emission_models, generate_sfh_basis, 
                            generate_metallicity_distribution, 
                            generate_constant_R, GalaxyBasis, CombinedBasis,
                            calculate_muv, draw_from_hypercube)
# Filters
# ---------------------------------------------------------------
# all medium and wide band filters for JWST NIRCam
filter_codes = [
    "JWST/NIRCam.F070W", "JWST/NIRCam.F090W", "JWST/NIRCam.F115W", "JWST/NIRCam.F140M",
    "JWST/NIRCam.F150W", "JWST/NIRCam.F162M", "JWST/NIRCam.F182M", "JWST/NIRCam.F200W",
    "JWST/NIRCam.F210M", "JWST/NIRCam.F250M", "JWST/NIRCam.F277W", "JWST/NIRCam.F300M",
    "JWST/NIRCam.F335M", "JWST/NIRCam.F356W", "JWST/NIRCam.F360M", "JWST/NIRCam.F410M",
    "JWST/NIRCam.F430M", "JWST/NIRCam.F444W", "JWST/NIRCam.F460M", "JWST/NIRCam.F480M",
]

# Consistent wavelength grid for both SPS grids and filters
new_wav = generate_constant_R(R=300)

filterset = FilterCollection(filter_codes, new_lam=new_wav)
instrument = Instrument('JWST', filters=filterset)



In [4]:
# params
Nmodels = 1e3
redshift = (5, 12)
masses = (5, 10)
weights = (0, 1)
max_redshift = 20 # gives maximum age of SFH at a given redshift
cosmo = Planck18 # cosmology to use for age calculations

# ---------------------------------------------------------------
# Pop II

tau_v = (0.0, 2.0)
log_zmet = (-3, 0.3)

# SFH
sfh_type = SFH.LogNormal
tau = (0.05, 2.5)
peak_age = (-1, 1) # normalized to maximum age of the universe at that redshift
sfh_param_units = [None, None]

# ---------------------------------------------------------------
# Pop III

min_age_popIII = (0.00, 29) # Myr
max_age_popIII = (0.01, 30) # Myr

# ---------------------------------------------------------------

# Generate the grid

full_params = {
    'redshift': redshift,
    'masses': masses,
    'weights': weights,
    'tau_v': tau_v,
    'log_zmet': log_zmet,
    'tau': tau,
    'peak_age': peak_age,
    'min_age_popIII': min_age_popIII,
    'max_age_popIII': max_age_popIII
}

# Generate the grid

param_grid = draw_from_hypercube(Nmodels, full_params, rng=42)

# Unpack the parameters

all_param_dict = {}
for i, key in enumerate(full_params.keys()):
    all_param_dict[key] = param_grid[:, i]


grid = Grid('bpass-2.2.1-bin_chabrier03-0.1,300.0_cloudy-c23.01-sps.hdf5',
            grid_dir='/home/tharvey/work/synthesizer_grids/',
            new_lam=new_wav)

# Metallicity 
Z_dists = [ZDist.DeltaConstant(log10metallicity=log_z) for log_z in all_param_dict['log_zmet']]

redshifts = np.array(all_param_dict['redshift'])

# Pop II SFH
sfh_param_arrays = np.vstack((all_param_dict['tau'], all_param_dict['peak_age'])).T
sfh_models, _ = generate_sfh_basis(
    sfh_type=sfh_type,
    sfh_param_names=['tau', 'peak_age_norm'],
    sfh_param_arrays=sfh_param_arrays,
    redshifts=redshifts,
    max_redshift=max_redshift,
    cosmo=cosmo,
    sfh_param_units=sfh_param_units,
    iterate_redshifts=False,
    calculate_min_age=False,
)

# Emission parameters
emission_model = TotalEmission(
    grid=grid,
    fesc=0.1,
    fesc_ly_alpha=0.1,
    dust_curve=PowerLaw(slope=-0.7),
    dust_emission_model=None,
)

# List of other varying or fixed parameters. Either a distribution to pull from or a list.
# Can be any parameter which can be property of emitter or galaxy and processed by the emission model.
galaxy_params={
    'tau_v': all_param_dict['tau_v'],
}

popII_basis = GalaxyBasis(
    model_name='Pop II',
    redshifts=redshifts,
    grid=grid,
    emission_model=emission_model,
    sfhs=sfh_models,
    cosmo=cosmo,
    instrument=instrument,
    metal_dists=Z_dists,
    galaxy_params=galaxy_params,
    redshift_dependent_sfh=True,
    params_to_ignore=['max_age'], # This is dependent on the redshift and should not be included in the basis
    build_grid=False,
)


# Second Population Model
# ---------------------------------------------------------------

popIII_grid = Grid('yggdrasil-1.3.3-PopIII_salpeter-10,1,500',
                    grid_dir='/home/tharvey/work/synthesizer_grids/',
                    read_lines=False, new_lam=new_wav)


# Pop III parameters

# SFH
sfh_array = np.vstack((all_param_dict['min_age_popIII'], all_param_dict['max_age_popIII'])).T

sfh_param_units = [Myr, Myr]
popIII_sfhs, _ = generate_sfh_basis(
    sfh_type=SFH.Constant,
    sfh_param_names=['min_age', 'max_age'],
    sfh_param_arrays=sfh_array,
    redshifts=redshifts,
    max_redshift=max_redshift,
    cosmo=cosmo,
    sfh_param_units=sfh_param_units,
    calculate_min_age=False,
    iterate_redshifts=False,
)

popIII_metal_dists = ZDist.DeltaConstant(metallicity=0)

if 'incident' not in grid.available_spectra:
    # Need a custom emission model for the Pop III grid if loading the nebular spectra. 
    popIII_emission_model = EmissionModel(
        label = 'Pop III',
        extract=popIII_grid.available_spectra[0],
        grid=popIII_grid,
        emitter='stellar'
    )
else:
    popIII_emission_model = IncidentEmission(
        grid=popIII_grid,
    )

popIII_basis = GalaxyBasis(
    model_name='Pop III',
    redshifts=redshifts,
    grid=popIII_grid,
    emission_model=popIII_emission_model,
    sfhs=popIII_sfhs,
    cosmo=cosmo,
    instrument=instrument,
    metal_dists=popIII_metal_dists,
    redshift_dependent_sfh=True,
    build_grid=False,
)

weights = np.array([np.array([i, 1-i]) for i in all_param_dict['weights']])



Creating SFHs: 1000it [00:00, 38526.16it/s]


Generating grid directly from provided parameter samples.


Creating SFHs: 1000it [00:00, 48449.30it/s]

Generating grid directly from provided parameter samples.


In [8]:
# --------------------------------------------------------------
# Combine the two population models

combined_basis = CombinedBasis(
    bases=[popII_basis, popIII_basis],
    total_stellar_masses=unyt_array(10**all_param_dict['masses'], units=Msun),
    base_emission_model_keys=['total', popIII_emission_model.label],
    combination_weights=weights,
    redshifts=redshifts,
    out_name='combined_basis_lhc',
    out_dir='/home/tharvey/work/output/',
    draw_parameter_combinations=False, # Since we have already drawn the parameters, we don't need to combine them again.
)

# Passing in extra analysis function to pipeline to calculate mUV. Any funciton could be passed in. 
combined_basis.process_bases(overwrite=False, mUV=(calculate_muv, cosmo))

combined_basis.create_grid(overwrite=True)


File /home/tharvey/work/output//combined_basis_lhc_Pop II.hdf5 already exists. Skipping.
File /home/tharvey/work/output//combined_basis_lhc_Pop III.hdf5 already exists. Skipping.
Emission model key for base Pop II: total
Emission model key for base Pop III: incident
[2.50292204e+35 1.03666107e+35] kg


NameError: name 'supp_params' is not defined